In [ ]:
import os
import neat
import pygame
import random
import copy
import time
import math
from ipynb.fs.full.gamedos import *

def eval_genomes(genomes, config):
    pygame.init()
    window_width, window_height = 700, 500
    window = pygame.display.set_mode((window_width, window_height))
    pickup_locations = [(50, 2), (10, 17), (40, 40), (45, 25)]
    
    
    for genome_id, genome in genomes:
        net = neat.nn.FeedForwardNetwork.create(genome, config)
        genome.fitness = 0
        
        # Define random control loss interval and initialize counter
        random_control_loss_interval = random.randint(100, 200)  # Example: between 100 and 200 iterations
        random_control_loss_counter = 0
        forced_commands_remaining = 0
        genome_start_time = time.time()

        game = Game(window, window_width, window_height, pickup_locations)

        #clock = pygame.time.Clock()
        while not game.game_info.finish:
            last_move_time = time.time()  # Track the last time the bus moved
            last_position = (game.bus.x, game.bus.y)  # Initial position of the bus
            current_time = time.time()
            #clock.tick(30)
            # Check if it's time for random control loss
            if random_control_loss_counter >= random_control_loss_interval and forced_commands_remaining == 0:
                forced_commands_remaining = 50  # Force 50 random commands
                random_control_loss_counter = 0  # Reset counter

            if forced_commands_remaining > 0:
                # Generate a random command
                direction = random.choice(["left", "right", "up", "down"])
                forced_commands_remaining -= 1
            else:
                # Use the neural network for decision making
                #output = net.activate((game.bus.x, game.bus.y, game.game_info.total_distance))
                #direction_index = output.index(max(output))
                #direction = ["left", "right", "up", "down"][direction_index]

                relative_x, relative_y = relative_position_to_closest_pickup(game.bus.x, game.bus.y, game.pickup_locations)

                if relative_x > 0:
                    direction = "right"
                elif relative_x < 0:
                    direction = "left"
                elif relative_y > 0:
                    direction = "up"
                elif relative_y < 0:
                    direction = "down"
                else:
                    output = net.activate((
                        game.bus.x, 
                        game.bus.y, 
                        game.game_info.total_distance,
                        relative_x,  # relative X position to the closest pickup
                        relative_y   # relative Y position to the closest pickup
                    ))
                

            #write code such thatt if value at output index = 3 is positive, then direction = ["right"]

            
            
            if current_time - genome_start_time > 100:
                break
            # Execute the command
            game.run(direction)

              # Check if the bus has moved
            if (game.bus.x, game.bus.y) != last_position:
                last_move_time = current_time  # Update the last move time
                last_position = (game.bus.x, game.bus.y)  # Update the last position

            # If the bus hasn't moved for more than 5 seconds, end the genome's evaluation
            if current_time - last_move_time > 5:
                break
            
            # Rendering and game update logic goes here...
            game.render()
            
            random_control_loss_counter += 1  # Increment the random control loss counter


            # Additional logic to determine when to end the genome's evaluation...

        # After exiting the loop, assign fitness
        genome.fitness = game.calculate_fitness()

def relative_position_to_closest_pickup(bus_x, bus_y, pickup_locations):
    if not pickup_locations:
        return float('inf'), float('inf')
    closest_pickup = min(pickup_locations, key=lambda p: math.hypot(p[0] - bus_x, p[1] - bus_y))
    relative_x = closest_pickup[0] - bus_x
    relative_y = closest_pickup[1] - bus_y
    return relative_x, relative_y

        
def run_neat(config_file):
    config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                         neat.DefaultSpeciesSet, neat.DefaultStagnation,
                         config_file)

    #p = neat.Checkpointer.restore_checkpoint('neat-checkpoint-0')
    p = neat.Population(config)
    p.add_reporter(neat.StdOutReporter(True))
    stats = neat.StatisticsReporter()
    p.add_reporter(stats)
    p.add_reporter(neat.Checkpointer(5))
    winner = p.run(eval_genomes, )
    with open("best.pickle", "wb") as f:
        pickle.dump(winner, f)

def replay_best_genome(config_file, genome_path="best_genome.pickle"):
    """
    Replays the best-performing genome from a NEAT run.
    
    :param config_file: Path to the NEAT configuration file.
    :param genome_path: Path to the saved best genome file.
    """
    # Load the NEAT configuration
    config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                         neat.DefaultSpeciesSet, neat.DefaultStagnation, config_file)
    
    # Load the saved best genome
    with open(genome_path, "rb") as f:
        genome = pickle.load(f)
    
    # Convert the genome into a neural network
    net = neat.nn.FeedForwardNetwork.create(genome, config)
    
    # Set up the game environment
    pygame.init()
    window_width, window_height = 700, 500
    window = pygame.display.set_mode((window_width, window_height))
    pickup_locations = [(50, 2), (40, 17), (40, 40), (45, 25)]  # Adjust to match your game setup
    
    # Initialize the game with the same parameters as in eval_genomes
    game = Game(window, window_width, window_height, pickup_locations)
    
    run = True
    while run and not game.game_info.finish:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                run = False
                pygame.quit()
                quit()
        
        # Determine the position of the closest pickup to guide the bus
        relative_x, relative_y = relative_position_to_closest_pickup(game.bus.x, game.bus.y, game.pickup_locations)
        
        # Get the neural network's output
        output = net.activate((game.bus.x, game.bus.y, game.game_info.total_distance, relative_x, relative_y))
        
        # Determine direction based on neural network output
        direction_index = output.index(max(output))
        direction = ["left", "right", "up", "down"][direction_index]
        
        # Run the game for one frame
        game.run(direction)
        
        # Render the current game state
        game.render()
        
        pygame.time.delay(100)  # Add a delay to make the game visually followable

if __name__ == "__main__":
    config_path = os.path.join(os.getcwd(), 'config-feedforward.txt')
    run_neat(config_path)
    genome_path = os.path.join(os.getcwd(), 'best.pickle')  # Adjust path as necessary
    #replay_best_genome(config_path, genome_path)


pygame 2.5.2 (SDL 2.28.3, Python 3.11.7)
Hello from the pygame community. https://www.pygame.org/contribute.html

 ****** Running generation 0 ****** 

134.0
175.07141400011596
(40, 200)
310.1886116991581
25.495097567963924
(60, 20)
-12.79356624028344
266.1766330841233
(10, 290)
-87.0
335.0373113550191
(40, 360)
Student picked up at (10, 17).
387.0
85.14693182963201
(40, 110)
-220.84682042344292
459.6194077712559
(150, 470)
Student picked up at (10, 17).
354.0
235.05318547086316
(40, 260)
